In [94]:
!pip install seaborn
!pip install dataframe_image

You should consider upgrading via the 'C:\Users\I\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\I\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [95]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
import dataframe_image as dfi
import numpy as np


In [103]:
DETAIL = "all"

DETAIL_MAP = {"cont0": "내용1 – 주제의 명료성", "cont1": "내용2 – 근거의 적절성", "cont3": "내용3 – 프롬프트 독해력",
              "exp0": "표현1 – 문법의 적절성", "exp1": "표현2 – 단어 사용의 적절성", "exp2": "표현3 – 문장 표현의 적절성",
              "org0": "구성1 – 문단 내 구조의 적절성", "org1": "구성2 – 문단 간 구조의 적절성", "org2": "구성3 – 구조의 일관성",
              "org3": "구성4 – 분량의 적절성"}

DIR_PATH = "./result"
CATEGORY = "글짓기"

In [104]:
df_essay = pd.read_excel(f"preprocess/{CATEGORY}_val.xlsx")
df_result_essay = pd.read_excel("result.xlsx")

In [98]:
# plt 한글깨짐 방지
plt.rc('font', family='Malgun Gothic')

df_label = df_essay
df_preds = df_result_essay

df_scatter = pd.DataFrame()
df_temp = df_label.drop(df_label.columns[0], axis=1)

df_scatter["label"] = df_temp.sum(axis=1)

df_scatter["pred"] = df_preds.sum(axis=1)

test = df_scatter["label"]
preds = df_scatter["pred"]
score_dict = {}

for detail in DETAIL_MAP.keys():
    print(detail)
    score_dict = {}
    test = df_label[detail]
    print(test)
    preds = df_preds[detail]
    print(preds)

# classification_report
report = classification_report(test, preds, output_dict=True)

report_df = pd.DataFrame(report)
dfi.export(report_df, f'{DIR_PATH}/result_report_{DETAIL}.png', max_cols=-1, max_rows=-1)
print(f"{CATEGORY}+{DETAIL} report SAVE")

C:\Users\I\AppData\Local\Temp\ipykernel_29264\1812403888.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_scatter["pred"] = df_preds.sum(axis=1)
C:\Users\I\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\I\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\I\AppData\R

글짓기+all report SAVE


In [99]:
preds

0     24
1     26
2     27
3     21
4     27
      ..
84    27
85    28
86    24
87    27
88    27
Name: pred, Length: 89, dtype: int64

In [100]:
# labels=[5, 6, 7, 8, 9]
print(pd.unique(test))
print(pd.unique(preds))
print(preds)
labels = np.concatenate((pd.unique(test), pd.unique(preds)))
labels = np.unique(labels)
labels = np.sort(labels)
print(labels)

[26 27 25 19 24 11 21 22 23 20 30 28 29]
[24 26 27 21 20 22 23 28 25 29 30]
0     24
1     26
2     27
3     21
4     27
      ..
84    27
85    28
86    24
87    27
88    27
Name: pred, Length: 89, dtype: int64
[11 19 20 21 22 23 24 25 26 27 28 29 30]


In [101]:
# confusion matrix
cm = confusion_matrix(test, preds, labels=labels)
ax = sns.heatmap(cm, annot=True, cmap='YlGn_r')
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
# plt.title(model_names[detail])
plt.title(DETAIL)
plt.savefig(f'{DIR_PATH}/result_confusion_matrix_{DETAIL}.png')
print(f"{CATEGORY}+{DETAIL} cm SAVE")
plt.clf()

글짓기+all cm SAVE


<Figure size 640x480 with 0 Axes>

In [102]:
# score
score_dict["accuracy_score"] = '{:.4f}'.format(accuracy_score(test, preds))
score_dict["f1_score(micro)"] = '{:.4f}'.format(f1_score(test, preds, average="micro"))
score_dict["f1_score(macro)"] = '{:.4f}'.format(f1_score(test, preds, average="macro"))
score_dict["f1_score(weighted)"] = '{:.4f}'.format(f1_score(test, preds, average="weighted"))

label_binarizer = LabelBinarizer().fit(test)
test = label_binarizer.transform(test)
preds = label_binarizer.transform(preds)
auc = roc_auc_score(
    test,
    preds,
    multi_class="ovr",
    average="micro",
)
score_dict["auc-ovr(mircro)"] = '{:.4f}'.format(auc)

score_df = pd.DataFrame(score_dict, index=[0])
score_df = score_df.style.set_table_styles([
    {'selector': 'th',
     'props': [('text-align', 'center')]},
    {'selector': 'td',
     'props': [('text-align', 'center')]}
])
dfi.export(score_df, f'{DIR_PATH}/result_score_{DETAIL}.png', max_cols=-1, max_rows=-1)
print(f"{CATEGORY}+{DETAIL} score SAVE")


글짓기+all score SAVE
